In [1]:

from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact

In [2]:
def wrangle(filepath):
    
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Get place name
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)
    
    #Drop high null value column
    df.drop(columns = ['floor' , 'expenses'], inplace = True)
    
    #High and low cardinality
    df.drop(columns = ['operation' , 'property_type' , 'currency' , 'properati_url'], inplace = True)
    
    #Drop Leakage
    df.drop(columns = ['price','price_aprox_local_currency','price_per_m2', 'price_usd_per_m2'], inplace = True)
    
    #Drop column with multicollinearity
    df.drop(columns = ['surface_total_in_m2', 'rooms'], inplace = True)
    

    
    return df

In [3]:
#Bringing in more files
files = glob(R"C:\Users\HP\Desktop\buenos-aires-real-estate-*.csv")

In [4]:
files

[]

In [5]:
df = [wrangle(file) for file in files]

In [6]:
df = pd.concat(df, ignore_index = True)

ValueError: No objects to concatenate

In [ ]:
plt.hist(df["surface_covered_in_m2"])
plt.xlabel("Area [sq meters]")
plt.title("Distribution of Apartment Sizes");

In [ ]:
plt.scatter(x= df["surface_covered_in_m2"], y = df["price_aprox_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title('Buenos Aires: Price vs Area')

plt.show()

In [ ]:
#spilting

X_train = df[['surface_covered_in_m2' ,'lat', 'lon','neighborhood']]

y_train = df["price_aprox_usd"]


In [ ]:
#Baseline prediction
y_pred_baseline =[y_train.mean()]* len(y_train)
y_pred_baseline[ :5]

In [ ]:
print("Mean apt price:", round(y_train.mean(), 2))

print("Baseline MAE:", mean_absolute_error(y_train, y_pred_baseline))

In [ ]:
#Building model

model = make_pipeline( 
    OneHotEncoder(use_cat_names = True), SimpleImputer(), Ridge()
)


model.fit(X_train, y_train)


In [ ]:
X_train.head()

In [ ]:
#Training
y_pred_training = model.predict(X_train)
print("Training MAE:", mean_absolute_error( y_train, y_pred_training))

In [ ]:
def make_prediction(area, lat, lon, neighborhood):
    data= {"surface_covered_in_m2" : area,
           'lat' : lat,
           'lon' : lon,
           'neighborhood' : neighborhood}
    df = pd.DataFrame(data, index =[0])
    
    prediction = model.predict(df).round(2)[0]
    return f"Predicted apartment price: ${prediction}"
   

In [ ]:
make_prediction(32, -22.60, -45.46, "flores")